In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
from sqlalchemy import create_engine
from pysandag.database import get_connection_string

db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

### 1. Get output of sr13

### get output of sr13: housing unit change grouped by jurisdiction

In [ ]:
sr13_units_total_sql  = '''WITH hs2050 AS
(SELECT y.City, x.increment, sum([hs]) AS hu
    FROM [regional_forecast].[sr13_final].[capacity] x
    inner join [regional_forecast].[sr13_final].[mgra13] AS y ON x.mgra = y.mgra
    WHERE x.scenario = 0 and x.increment in (2050)  and hs > 0
    GROUP BY y.City, x.increment), hs2012 AS
(SELECT y.City, x.increment, sum([hs]) AS hu
    FROM [regional_forecast].[sr13_final].[capacity] x
    inner join [regional_forecast].[sr13_final].[mgra13] AS y ON x.mgra = y.mgra
    WHERE x.scenario = 0 and x.increment in (2012) and hs > 0
    GROUP BY y.City, x.increment)
SELECT hs2012.City,hs2050.hu,hs2012.hu , hs2050.hu  - hs2012.hu as hu_change
from hs2012 
join hs2050 on hs2012.City = hs2050.City
order by hs2012.City'''
sr13_units_total = pd.read_sql(sr13_units_total_sql,mssql_engine)
sr13_units_total

In [ ]:
sr13_sql_match_12to15 = '''
     SELECT c.City, c.yr_from,ahu, c.yr_to,bhu, c.hu_change,hu_change_w_neg
    FROM (SELECT  a.City, a.increment AS yr_from, b.increment AS yr_to
     ,b.hu - a.hu as hu_change_w_neg
    ,CASE WHEN b.hu - a.hu > 0 THEN b.hu - a.hu ELSE 0 END AS hu_change,
    b.hu as bhu, a.hu as ahu
    FROM (SELECT  y.City, x.increment, sum(x.hs) AS hu
    FROM [regional_forecast].[sr13_final].[capacity] AS x
    inner join [regional_forecast].[sr13_final].[mgra13] AS y ON x.mgra = y.mgra
    WHERE x.increment in (2012)  and scenario = 0
    GROUP BY  y.City, x.increment) AS a
    inner join 
    (SELECT y.City,x.increment, sum(x.hs) AS hu
    FROM [regional_forecast].[sr13_final].[capacity] AS x
    inner join [regional_forecast].[sr13_final].[mgra13] AS y ON x.mgra = y.mgra
    WHERE x.increment in (2015) and scenario = 0
    GROUP BY  y.City, x.increment) AS b
    ON a.City = b.City and a.increment = b.increment-3) AS c
    ORDER BY yr_from, City
'''
sr13_units_by_jur_a = pd.read_sql(sr13_sql_match_12to15,mssql_engine)

In [ ]:
sr13_sql_match_15to50 = '''
      SELECT c.City, c.yr_from,ahu, c.yr_to,bhu, c.hu_change,hu_change_w_neg
    FROM (SELECT  a.City, a.increment AS yr_from, b.increment AS yr_to
     ,b.hu - a.hu as hu_change_w_neg
    ,CASE WHEN b.hu - a.hu > 0 THEN b.hu - a.hu ELSE 0 END AS hu_change,
    b.hu as bhu, a.hu as ahu
    FROM (SELECT  y.City, x.increment, sum(x.hs) AS hu
    FROM [regional_forecast].[sr13_final].[capacity] AS x
    inner join [regional_forecast].[sr13_final].[mgra13] AS y ON x.mgra = y.mgra
    WHERE x.increment in (2015,2020, 2025, 2030, 2035, 2040, 2045, 2050)  and scenario = 0
    GROUP BY  y.City, x.increment) AS a
    inner join 
    (SELECT y.City,x.increment, sum(x.hs) AS hu
    FROM [regional_forecast].[sr13_final].[capacity] AS x
    inner join [regional_forecast].[sr13_final].[mgra13] AS y ON x.mgra = y.mgra
    WHERE x.increment in (2015,2020, 2025, 2030, 2035, 2040, 2045, 2050) and scenario = 0
    GROUP BY  y.City, x.increment) AS b
    ON a.City = b.City and a.increment = b.increment-5) AS c
    ORDER BY yr_from, City
'''
sr13_units_by_jur_b = pd.read_sql(sr13_sql_match_15to50,mssql_engine)

In [ ]:
sr13_units_by_jur = pd.concat([sr13_units_by_jur_a,sr13_units_by_jur_b])

### check results

In [ ]:
sr13_units_by_jur.loc[sr13_units_by_jur.City==4]

In [ ]:
sr13_units_by_jur.loc[sr13_units_by_jur.City==4].hu_change.sum()

In [ ]:
sr13_units_by_jur.loc[sr13_units_by_jur.City==4].hu_change_w_neg.sum()

In [ ]:
sr13_units = pd.DataFrame({'hs_added': sr13_units_by_jur.
                           groupby(["City"]).
                           hu_change.sum()}).reset_index()

In [ ]:
sr13_units

In [ ]:
sr13_units.hs_added.sum()

### capacity from [sr13_final].[capacity] (includes sched dev)

In [ ]:
sr13_cap_by_jur_sql = '''
    SELECT y.City,sum([cap_hs]) AS cap
    FROM [regional_forecast].[sr13_final].[capacity] x
    inner join [regional_forecast].[sr13_final].[mgra13] AS y ON x.mgra = y.mgra
    WHERE x.scenario = 0 and x.increment in (2012) 
    GROUP BY y.City, x.increment
    ORDER BY y.City'''

In [ ]:
sr13_cap_by_jur =  pd.read_sql(sr13_cap_by_jur_sql,mssql_engine,index_col='City')

In [ ]:
# sr13_cap_by_jur.head()

### set increment as 'yr_to'

In [ ]:
sr13_units_by_jur.loc[sr13_units_by_jur.City==8]

In [ ]:
sr13_units_by_jur['increment'] = sr13_units_by_jur['yr_to']

In [ ]:
sr13_units_by_jur = sr13_units_by_jur.drop(['yr_from', 'yr_to'],axis =1)

In [ ]:
# sr13_units_by_jur.head()

### join sr13 unit change with capacity

In [ ]:
units_by_jur_inc = sr13_units_by_jur.set_index('City').join(sr13_cap_by_jur)

In [ ]:
units_by_jur_inc.head()

### add starting year as 2012 with unit change as 0 (for plotting)

In [ ]:
start_year = units_by_jur_inc.loc[units_by_jur_inc.increment==2025].copy()

In [ ]:
# start_year

In [ ]:
start_year['increment'] = 2012
start_year['hu_change'] = 0
start_year['hu_change_neg'] = 0

In [ ]:
units_by_jur_inc = pd.concat([start_year,units_by_jur_inc])

In [ ]:
units_by_jur_inc.loc[units_by_jur_inc.City==8]

### pivot for plotting so jurisdictions are columns and increments are rows with new units

In [ ]:
units_by_jur_inc.reset_index(inplace=True, drop=False)

In [ ]:
units_by_jur_pivot = units_by_jur_inc.pivot\
(index='increment', columns='City', values='hu_change_neg').\
reset_index().rename_axis(None, axis=1)
units_by_jur_pivot.fillna(0,inplace=True)
units_by_jur_pivot.set_index('increment',inplace=True)

In [ ]:
# units_by_jur_pivot.loc[sr13_units_by_jur.City==8]

In [ ]:
units_by_jur_pivot.head()

### pivot for plotting so jurisdictions are columns and increments are rows with capacity

In [ ]:
cap_by_jur_pivot = units_by_jur_inc.pivot\
(index='increment', columns='City', values='cap').\
reset_index().rename_axis(None, axis=1)
cap_by_jur_pivot.fillna(0,inplace=True)
cap_by_jur_pivot.set_index('increment',inplace=True)

### calculate cumulative sum of units added by jurisdiction

In [ ]:
units_by_jur_pivot = units_by_jur_pivot.cumsum()

### plot results

#### for each jurisdiction join dataframe of new units and capacity

#### calculate remaining capacity by subtracting new units from starting capacity

In [ ]:
df_plot

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=10, ncols=2)
for j, jur in enumerate(units_by_jur_inc.City.unique().tolist()):
    df_units_added = units_by_jur_pivot[[jur]]
    df_capacity = cap_by_jur_pivot[[jur]]
    df_plot = df_units_added.join(df_capacity, lsuffix='_new', rsuffix='_capacity')
    hs_column = str(jur) + '_new'
    cap_column = str(jur) + '_capacity'
    cap_plot_column = str(jur) + '_cap'
    df_plot[cap_plot_column] = df_plot[cap_column] - df_plot[hs_column]
    del  df_plot[cap_column]
    df_plot.plot(style='.-',ax=axes.flat[j],figsize=(10,32))
   # df_plot.plot(kind='bar',ax=axes.flat[j],figsize=(10,32)))

In [ ]:
df_plot.plot(kind='bar')

In [ ]:
jur = 'Encinitas'
jur_id = 6
confluence_pg_cap = 2709
jur = 'Imperial Beach'
jur_id = 8
confluence_pg_cap = 1840

In [ ]:
print("{} housing unit change and remaining capacity".format(jur))

In [ ]:
df_units_added = units_by_jur_pivot[[jur_id]]
df_units_added
df_capacity = cap_by_jur_pivot[[jur_id]]
df_capacity
df_plot = df_units_added.join(df_capacity, lsuffix='_hs_change', rsuffix='_capacity')

In [ ]:
df_plot

In [ ]:
df_units_added = units_by_jur_pivot[[jur_id]]
df_capacity = cap_by_jur_pivot[[jur_id]]
df_plot = df_units_added.join(df_capacity, lsuffix='_hs_change', rsuffix='_capacity')
hs_column = str(jur_id) + '_hs_change'
cap_column = str(jur_id) + '_capacity'
cap_plot_column = str(jur_id) + '_cap'
df_plot[cap_plot_column] = df_plot[cap_column] - df_plot[hs_column]
del  df_plot[cap_column]
df_plot.plot(style='.-',title=jur)

In [ ]:
df_plot

## previous version

In [ ]:
#  missing 2015 increment

sr13_sql_match_12to20 = '''
    SELECT c.jurisdiction, c.jurisdiction_id, c.yr_from, c.yr_to, c.hu_change,hu_change_neg
    FROM (SELECT a.jurisdiction, a.jurisdiction_id, a.yr_id AS yr_from, b.yr_id AS yr_to
    ,b.hu - a.hu as hu_change_neg
    ,CASE WHEN b.hu - a.hu > 0 THEN b.hu - a.hu ELSE 0 END AS hu_change
    FROM (SELECT y.jurisdiction, y.jurisdiction_id, x.yr_id, sum(x.units) AS hu
    FROM [demographic_warehouse].[fact].[housing] AS x
    inner join [demographic_warehouse].[dim].[mgra_denormalize] AS y ON x.mgra_id = y.mgra_id
    WHERE x.datasource_id = 13 and x.yr_id in (2012, 2020)
    GROUP BY y.jurisdiction, y.jurisdiction_id, x.yr_id) AS a
    inner join 
    (SELECT y.jurisdiction,y.jurisdiction_id,x.yr_id, sum(x.units) AS hu
    FROM [demographic_warehouse].[fact].[housing] AS x
    inner join [demographic_warehouse].[dim].[mgra_denormalize] AS y ON x.mgra_id = y.mgra_id
    WHERE x.datasource_id = 13 and x.yr_id in (2012, 2020)
    GROUP BY y.jurisdiction, y.jurisdiction_id, x.yr_id) AS b
    ON a.jurisdiction_id = b.jurisdiction_id and a.yr_id = b.yr_id-8) AS c
    ORDER BY yr_from, jurisdiction_id
'''
sr13_units_by_jur_a = pd.read_sql(sr13_sql_match_12to20,mssql_engine)
sr13_sql_match_20to50 = '''
    SELECT c.jurisdiction, c.jurisdiction_id, c.yr_from, c.yr_to, c.hu_change,hu_change_neg
    FROM (SELECT a.jurisdiction, a.jurisdiction_id, a.yr_id AS yr_from, b.yr_id AS yr_to
    ,b.hu - a.hu as hu_change_neg
    ,CASE WHEN b.hu - a.hu > 0 THEN b.hu - a.hu ELSE 0 END AS hu_change
    FROM (SELECT y.jurisdiction, y.jurisdiction_id, x.yr_id, sum(x.units) AS hu
    FROM [demographic_warehouse].[fact].[housing] AS x
    inner join [demographic_warehouse].[dim].[mgra_denormalize] AS y ON x.mgra_id = y.mgra_id
    WHERE x.datasource_id = 13 and x.yr_id in (2020, 2025, 2030, 2035, 2040, 2045, 2050)
    GROUP BY y.jurisdiction, y.jurisdiction_id, x.yr_id) AS a
    inner join 
    (SELECT y.jurisdiction,y.jurisdiction_id,x.yr_id, sum(x.units) AS hu
    FROM [demographic_warehouse].[fact].[housing] AS x
    inner join [demographic_warehouse].[dim].[mgra_denormalize] AS y ON x.mgra_id = y.mgra_id
    WHERE x.datasource_id = 13 and x.yr_id in (2020, 2025, 2030, 2035, 2040, 2045, 2050)
    GROUP BY y.jurisdiction, y.jurisdiction_id, x.yr_id) AS b
    ON a.jurisdiction_id = b.jurisdiction_id and a.yr_id = b.yr_id-5) AS c
    ORDER BY yr_from, jurisdiction_id
'''
sr13_units_by_jur_b = pd.read_sql(sr13_sql_match_20to50,mssql_engine)

sr13_units_by_jur = pd.concat([sr13_units_by_jur_a,sr13_units_by_jur_b])